In [4]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=41dea7c6ea5d620ce1c964d05b7b59b00c67cf5d74ae0cff251bb7795d81f603
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# Еще буду исправлять

In [43]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, length, avg, lower, regexp_replace
import re

In [54]:
# Создание SparkSession
spark = SparkSession.builder.getOrCreate()

# Загрузка данных из файла
file_path = "wiki.txt"
df = spark.read.option("delimiter", "\t").csv(file_path, header=False).toDF("url", "title", "text")

# Разбивка текста на слова
words_df = df.select(explode(split(col("text"), "\\s+")) \
                     .alias("word"))

In [55]:
# 1. Найти самое длинное слово
longest_word = words_df \
    .filter(col("word").rlike(r"^[a-zA-Zа-яА-Я]+$")) \
    .withColumn("length", length(col("word"))) \
    .orderBy(col("length").desc()) \
    .select("word") \
    .first()

print(f"Самое длинное слово: {longest_word['word']}")

Самое длинное слово: никотинамидадениндинуклеотида


In [57]:
# 2. Найти среднюю длину слов
average_word_length = words_df.withColumn("length", length(col("word"))) \
    .filter(col("word").rlike(r"^[a-zA-Zа-яА-Я]+$")) \
    .agg(avg(col("length")).alias("average_length")) \
    .first()

print(f"Средняя длина слов: {average_word_length['average_length']}")

Средняя длина слов: 6.198729955995877


In [58]:
# 3. Найти самое частоупотребляемое слово, состоящее из латинских букв
latin_words_df = words_df.filter(col("word").rlike("^[a-zA-Z]{3,}")) \
                         .filter(col("word").rlike("[^IXV]"))

most_common_latin_word = latin_words_df.groupBy("word") \
    .count() \
    .orderBy(col("count").desc()) \
    .first()

print(f"Самое частоупотребляемое латинское слово: {most_common_latin_word['word']}")

Самое частоупотребляемое латинское слово: The


In [68]:
# 4. Слова, которые более чем в половине случаев начинаются с большой буквы и встречаются больше 10 раз


In [61]:
# 5. Определить устойчивые сокращения вида пр., др.
abbreviations_df = words_df.filter(col("word").rlike("^[а-я]{1,3}\.$"))
abbreviations_df.groupBy("word") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(truncate=False)

+----+-----+
|word|count|
+----+-----+
|тыс.|4211 |
|г.  |2806 |
|лет.|1562 |
|др. |1276 |
|год.|1055 |
|гг. |961  |
|н.  |943  |
|им. |906  |
|век.|876  |
|чел.|759  |
|э.  |579  |
|т.  |439  |
|в.  |358  |
|руб.|351  |
|км. |350  |
|с.  |337  |
|ст. |331  |
|св. |313  |
|нет.|311  |
|см. |302  |
+----+-----+
only showing top 20 rows



In [62]:
# 6. Определить устойчивые сокращения вида т.п., н.э.
complex_abbreviations_df = words_df.filter(col("word").rlike("^[а-я]{1,3}\.[а-я]{1,3}\.$"))
complex_abbreviations_df.groupBy("word") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(truncate=False)

+-------+-----+
|word   |count|
+-------+-----+
|т.е.   |60   |
|н.э.   |44   |
|т.д.   |38   |
|л.с.   |24   |
|т.к.   |24   |
|кв.м.  |23   |
|т.п.   |17   |
|т.ч.   |17   |
|с.ш.   |17   |
|т.н.   |17   |
|ю.ш.   |15   |
|р.п.   |10   |
|кв.км. |7    |
|г.г.   |7    |
|рт.ст. |6    |
|ед.ч.  |6    |
|куб.см.|6    |
|в.д.   |5    |
|и.о.   |5    |
|ст.ст. |5    |
+-------+-----+
only showing top 20 rows



In [80]:
#Загружаем мужские и женские имена на русском языке
with open("data/female_names_rus.txt") as f:
  female_names = [name.replace("\n", "") for name in f.readlines()]

with open("data/male_names_rus.txt") as f:
  male_names = [name.replace("\n", "") for name in f.readlines()]

In [84]:
# 7. Найти имена, употребляющиеся в статьях
names_df = words_df.filter(col("word").isin(female_names) | (col("word").isin(male_names))) \
    .groupBy("word") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(truncate=False)

+----------+-----+
|word      |count|
+----------+-----+
|          |12595|
|Александр |1188 |
|Александра|847  |
|Владимир  |817  |
|Петра     |760  |
|Николай   |679  |
|Пётр      |661  |
|Сергей    |601  |
|Михаил    |583  |
|Джон      |544  |
|Иван      |503  |
|Владимира |454  |
|Юрий      |407  |
|Виктор    |378  |
|Борис     |376  |
|Алексей   |369  |
|Фридрих   |334  |
|Андрей    |320  |
|Дмитрий   |319  |
|Ленина    |306  |
+----------+-----+
only showing top 20 rows



In [ ]:
# Завершение работы SparkSession
spark.stop()